In [99]:
import numpy as np

## ufunc函数

ufunc 是 universal function 的缩写, 万能函数

In [100]:
x = np.linspace(0, 2*np.pi, 10)
y = np.sin(x)
y

array([  0.00000000e+00,   6.42787610e-01,   9.84807753e-01,
         8.66025404e-01,   3.42020143e-01,  -3.42020143e-01,
        -8.66025404e-01,  -9.84807753e-01,  -6.42787610e-01,
        -2.44929360e-16])

out参数可以指定输出数组

In [101]:
t = np.sin(x, out=x)
t is x

True

对于数组的计算, math.sin() 比 math.sin()快, 对于单个数值的计算,math.sin()却比math.sin() 快 

In [102]:
import math

x = [i * 0.001 for i in xrange(1000000)]

def sin_math(x):
    for i, t in enumerate(x):
        x[i] = math.sin(t)

def sin_numpy(x):
    np.sin(x, x)

def sin_numpy_loop(x):
    for i, t in enumerate(x):
        x[i] = np.sin(t)

xl = x[:]
%time sin_math(x)

xa = np.array(x)
%time sin_numpy(xa)

xl = x[:]
%time sin_numpy_loop(x)

Wall time: 218 ms
Wall time: 16 ms
Wall time: 1.11 s


In [103]:
%C type(math.sin(0.5)); type(np.sin(0.5))

type(math.sin(0.5))  type(np.sin(0.5))
-------------------  -----------------
float                numpy.float64    


item()方法获取数组中的单个元素, 将其转化成标准的Python数值类型

In [104]:
a = np.arange(6.0).reshape(2, 3)
print "a:\n{} \n ".format(a)
print a.item(1, 2), type(a.item(1, 2)), type(a[1, 2])

a:
[[ 0.  1.  2.]
 [ 3.  4.  5.]] 
 
5.0 <type 'float'> <type 'numpy.float64'>


### 四则运算

numpy 提供了许多ufunc函数: 

add,subtract,multiply,

divide,true_divede,floor_divede,

negative,power,

remainder,mod

In [105]:
a = np.arange(0, 4)
b = np.arange(1, 5)
%C a; b
np.add(a, b)

     a             b      
------------  ------------
[0, 1, 2, 3]  [1, 2, 3, 4]


array([1, 3, 5, 7])

In [106]:
# 第三个参数省略了关键字out
np.add(a, b, a)
a

array([1, 3, 5, 7])

### 比较和布尔运算

比较运算符参与的运算,将返回布尔数组, 比较运算符对应的ufunc函数有哪些: equal, not_equal, less, less_equal, greater, greater_equal

In [107]:
np.array([1, 2, 3]) < np.array([3, 2, 1])

array([ True, False, False], dtype=bool)

python 中的布尔运算用关键字 and, or, not 表示, 但是数组中的布尔运算只能通过相应的ufunc函数进行, 这些函数名都已logical_开头

In [108]:
a = np.arange(5)
b = np.arange(4, -1, -1)
%C a; b

       a                b       
---------------  ---------------
[0, 1, 2, 3, 4]  [4, 3, 2, 1, 0]


In [109]:
print a == b
print a > b
print np.logical_or(a == b, a > b)  # 和 a>=b 相同

[False False  True False False]
[False False False  True  True]
[False False  True  True  True]


In [110]:
# 对两个布尔数组使用 and, or, not 进行布尔运算, 将抛出ValueError异常
a == b and a > b

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [111]:
%C np.any(a == b);np.any(a > b); np.any(a == b) and np.any(a > b)

np.any(a == b)  np.any(a > b)  np.any(a == b) and np.any(a > b)
--------------  -------------  --------------------------------
True            True           True                            


以bitwise_开头的函数都是位运算函数

In [112]:
print a == b
print a > b

[False False  True False False]
[False False False  True  True]


In [113]:
print a == b
print a > b
(a == b) | (a > b)

[False False  True False False]
[False False False  True  True]


array([False, False,  True,  True,  True], dtype=bool)

In [114]:
~ np.arange(5)

array([-1, -2, -3, -4, -5])

### 自定义ufunc函数

frompyfunc() 将 标量函数 (普通函数)转化成 (万能函数)矢量函数

In [115]:
def triangle_wave(x, c, c0, hc):
    x = x - int(x) # 三角波的周期为1，因此只取x坐标的小数部分进行计算
    if x >= c: r = 0.0
    elif x < c0: r = x / c0 * hc
    else: r = (c-x) / (c-c0) * hc
    return r

In [116]:
triangle_ufunc1 = np.frompyfunc(triangle_wave, 4, 1)
# 4 是输入参数的个数, 1 是输出值的个数
y2 = triangle_ufunc1(x, 0.6, 0.4, 1.0)
y2

array([0.0, 0.0024999991666669165, 0.004999993333341332, ..., 0.0, 0.0, 0.0], dtype=object)

In [117]:
# triangle_ufunc1()所返回数组的元素类型是object
%C y2.dtype; y2.astype(np.float).dtype

 y2.dtype   y2.astype(np.float).dtype
----------  -------------------------
dtype('O')  dtype('float64')         


列表推导式实现类似于ufunc函数的功能

In [118]:
x = np.linspace(0, 2, 1000)
y1 = np.array([triangle_wave(t, 0.6, 0.4, 1.0) for t in x])

vectorize()也可实现frompyfunc的功能, 且提供调节数组类型的关键字参数otypes

In [119]:
triangle_ufunc2 = np.vectorize(triangle_wave, otypes=[np.float])
y3 = triangle_ufunc2(x, 0.6, 0.4, 1.0)

In [120]:
%C np.all(y1 == y2); np.all(y2 == y3)

np.all(y1 == y2)  np.all(y2 == y3)
----------------  ----------------
False             False           


### 广播

unfunc函数处理多个数组时会要求数组的形状相同, 形状不同时要进行广播处理

>首先要同维, 维数少的前面加 1 

>其次要同度, 每个维数选最大的度

>可以按行按列扩展维度

In [121]:
a = np.arange(0, 60, 10).reshape(-1, 1)
%C a; a.shape

  a     a.shape
------  -------
[[ 0],  (6, 1) 
 [10],         
 [20],         
 [30],         
 [40],         
 [50]]         


In [122]:
b = np.arange(0, 5)
%C b; b.shape

       b         b.shape
---------------  -------
[0, 1, 2, 3, 4]  (5,)   


In [123]:
c = a + b
%C c; c.shape

          c             c.shape
----------------------  -------
[[ 0,  1,  2,  3,  4],  (6, 5) 
 [10, 11, 12, 13, 14],         
 [20, 21, 22, 23, 24],         
 [30, 31, 32, 33, 34],         
 [40, 41, 42, 43, 44],         
 [50, 51, 52, 53, 54]]         


In [124]:
# 广播运算并不会隐性的改变数组的维度
%C b; b.shape

       b         b.shape
---------------  -------
[0, 1, 2, 3, 4]  (5,)   


In [125]:
# repeat()方法沿着指定的轴复制该轴的第一个元素
b_15 = b[None, :]
b_re = b_15.repeat(6, axis=0)
%C b_15; b_re

       b_15               b_re      
-----------------  -----------------
[[0, 1, 2, 3, 4]]  [[0, 1, 2, 3, 4],
                    [0, 1, 2, 3, 4],
                    [0, 1, 2, 3, 4],
                    [0, 1, 2, 3, 4],
                    [0, 1, 2, 3, 4],
                    [0, 1, 2, 3, 4]]


ogrid[]对象可以将切片转化为可广播的二维数组, 也即曲面的二维轴

In [126]:
x, y = np.ogrid[:5, :5]
%C x; y

  x            y        
-----  -----------------
[[0],  [[0, 1, 2, 3, 4]]
 [1],                   
 [2],                   
 [3],                   
 [4]]                   


mgrid[]对象将切片转化为可广播的二维数组, 也即曲面的平面坐标

In [127]:
x, y = np.mgrid[:5, :5]
%C x; y

        x                  y        
-----------------  -----------------
[[0, 0, 0, 0, 0],  [[0, 1, 2, 3, 4],
 [1, 1, 1, 1, 1],   [0, 1, 2, 3, 4],
 [2, 2, 2, 2, 2],   [0, 1, 2, 3, 4],
 [3, 3, 3, 3, 3],   [0, 1, 2, 3, 4],
 [4, 4, 4, 4, 4]]   [0, 1, 2, 3, 4]]


ogrid[]对象下标含有j时, j前的数字表示始终值之间的个数, 类似于linspace()的参数形式

In [128]:
x, y = np.ogrid[:1:4j, :1:3j]
%C x; y   

       x                  y          
---------------  --------------------
[[ 0.        ],  [[ 0. ,  0.5,  1. ]]
 [ 0.33333333],                      
 [ 0.66666667],                      
 [ 1.        ]]                      


In [157]:
from mpl_toolkits.mplot3d.axes3d import Axes3D
import pylab as pl

x, y = np.ogrid[-1:1:21j, -1:1:21j]
z = x**2 + y**2 

fig = pl.figure(figsize=(15, 5))
ax = fig.add_subplot(1, 2, 1, projection='3d')
surf = ax.plot_surface(x, y, z, rstride=1, cstride=1, \
                       cmap="coolwarm", linewidth=0.2)
pl.show()

数组支持特殊的下表对象 None

In [131]:
# a[None, :] == a.reshape(1, -1);  a[:, None] == a.reshape(-1, 1)
a = np.arange(4)
%C a[None, :]; a[:, None]

  a[None, :]    a[:, None]
--------------  ----------
[[0, 1, 2, 3]]  [[0],     
                 [1],     
                 [2],     
                 [3]]     


In [132]:
x = np.array([0, 1, 4, 10])    
y = np.array([2, 3, 8])   
x[None, :] + y[:, None]

array([[ 2,  3,  6, 12],
       [ 3,  4,  7, 13],
       [ 8,  9, 12, 18]])

np.ix_() 可以将一维数组转化成可广播的二维数组

In [133]:
gy, gx = np.ix_(y, x) 
%C gx; gy; gx + gy

        gx            gy        gx + gy      
------------------  -----  ------------------
[[ 0,  1,  4, 10]]  [[2],  [[ 2,  3,  6, 12],
                     [3],   [ 3,  4,  7, 13],
                     [8]]   [ 8,  9, 12, 18]]


np.broadcast_arrays 返回可广播数组广播后的数组

In [134]:
gx_bc, gy_bc = np.broadcast_arrays(gx, gy)
%C gx_bc; gy_bc

      gx_bc             gy_bc     
------------------  --------------
[[ 0,  1,  4, 10],  [[2, 2, 2, 2],
 [ 0,  1,  4, 10],   [3, 3, 3, 3],
 [ 0,  1,  4, 10]]   [8, 8, 8, 8]]


### ufunc的方法

ufunc 方法只对 两个输入一个输出的ufunc函数有效

reduce()会对参数序列中的元素进行累积

In [135]:
r1 = np.add.reduce([1, 2, 3])  # 1 + 2 + 3
r2 = np.add.reduce([[1, 2, 3], [4, 5, 6]], axis=1)  # (1+2+3),(4+5+6)
%C r1; r2

r1     r2   
--  --------
6   [ 6, 15]


accumulate()方法和reduce方法类似, 只是他返回的结果数组和输入数组形状相同,保存所有中间计算结果

a1 = np.add.accumulate([1, 2, 3])
a2 = np.add.accumulate([[1, 2, 3], [4, 5, 6]], axis=1)
%C a1; a2

 reduceeat() 方法计算 多组 reduce() 的结果, indices 参数可以指定一系列的起始位置和终止位置, indice是的每个元素都会计算一个值, 最后的计算结果和indices参数的长度相同

根据indices计算reduce()的reduce的参数: 正序取区间, 逆序取本身, 最后一个输出元素是所欲元素的值

In [136]:
a = np.array([1, 2, 3, 4])
result = np.add.reduceat(a, indices=[0, 1, 0, 2, 0, 3, 0])
result

array([ 1,  2,  3,  3,  6,  4, 10])

np.multiply.outer()方法计算列向量和行向量的外积,

np.multiply.outer() 方法计算列向量和行向量的外和

In [137]:
np.multiply.outer([1, 2, 3, 4, 5], [2, 3, 4])

array([[ 2,  3,  4],
       [ 4,  6,  8],
       [ 6,  9, 12],
       [ 8, 12, 16],
       [10, 15, 20]])

In [138]:
np.add.outer([1, 2, 3, 4, 5], [2, 3, 4])

array([[3, 4, 5],
       [4, 5, 6],
       [5, 6, 7],
       [6, 7, 8],
       [7, 8, 9]])